In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
# matplotlib 한글 폰트 출력코드
# 출처 : 데이터공방( https://kiddwannabe.blog.me)

import matplotlib
from matplotlib import font_manager, rc
import platform

try : 
    if platform.system() == 'Windows':
    # 윈도우인 경우
        font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
        rc('font', family=font_name)
    else:    
    # Mac 인 경우
        rc('font', family='AppleGothic')
except : 
    pass
matplotlib.rcParams['axes.unicode_minus'] = False  

In [3]:
weather=pd.read_csv('weather.csv')

weather.head(3)

,지점,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),전운량(10분위),현상번호(국내식)
0,184,2019-09-01 00:00,23.7,NaN,2.0,67,7.0,NaN
1,184,2019-09-01 01:00,23.7,NaN,2.1,67,10.0,NaN
2,184,2019-09-01 02:00,23.5,NaN,1.4,70,9.0,NaN


In [4]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4372 entries, 0 to 4371
Data columns (total 8 columns):
지점           4372 non-null int64
일시           4372 non-null object
기온(°C)       4372 non-null float64
강수량(mm)      943 non-null float64
풍속(m/s)      4326 non-null float64
습도(%)        4372 non-null int64
전운량(10분위)    4354 non-null float64
현상번호(국내식)    637 non-null float64
dtypes: float64(5), int64(2), object(1)
memory usage: 273.4+ KB


In [5]:
weather.isna().sum()

지점              0
일시              0
기온(°C)          0
강수량(mm)      3429
풍속(m/s)        46
습도(%)           0
전운량(10분위)      18
현상번호(국내식)    3735
dtype: int64

### 숫자를 지역명으로 바꾸기

In [25]:
### (https://data.kma.go.kr/data/grnd/selectAsosRltmList.do?pgmNo=36) -관측소 위도 경도도 써있음

weather['지점'] = ['jeju' if i == '184' else i for i in weather['지점']]      #제주  위도 : 33.51411 경도 : 126.52969
weather['지점'] = ['gosan' if i == '185' else i for i in weather['지점']]     # 고산 위도 : 33.29382 경도 : 126.16283
weather['지점'] = ['seongsan' if i == '188' else i for i in weather['지점']]  # 성산 위도 : 33.38677 경도 : 126.8802
weather['지점'] = ['po' if i == '189' else i for i in weather['지점']]        #서귀포 위도 : 33.24616 경도 : 126.5653
weather.head(10)

,지점,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),전운량(10분위),현상번호(국내식)
0,jeju,2019-09-01 00:00,23.7,NaN,2.0,67,7.0,NaN
1,jeju,2019-09-01 01:00,23.7,NaN,2.1,67,10.0,NaN
2,jeju,2019-09-01 02:00,23.5,NaN,1.4,70,9.0,NaN
3,jeju,2019-09-01 03:00,23.4,NaN,1.1,68,8.0,NaN
4,jeju,2019-09-01 04:00,23.4,NaN,1.6,69,10.0,NaN
5,jeju,2019-09-01 05:00,23.3,NaN,1.2,71,10.0,NaN
6,jeju,2019-09-01 06:00,23.4,NaN,1.2,73,10.0,NaN
7,jeju,2019-09-01 07:00,23.5,0.0,0.9,71,10.0,1.0
8,jeju,2019-09-01 08:00,24.0,0.0,0.7,69,10.0,1.0
9,jeju,2019-09-01 09:00,24.9,0.0,1.3,68,10.0,1.0


In [67]:
# 시간과 일자 분리

weather['time'] = [int(i.split(' ')[1].split(':')[0]) for i in weather['일시']]
weather['일시'] = [i.split(' ')[0] for i in weather['일시']]

In [242]:
#9월 데이터(오전, 오후)
weather_9m=weather[(weather['일시']>='2019-09-01') & (weather['일시']<='2019-09-30')]
weather_9m

cols =weather.columns.tolist()
cols = cols[0:2] +cols[-1:]+cols[2:8]

#열 재배치 
weather_9m=weather_9m[cols]

#na 값에 0.0
weather_9m=weather_9m.fillna(0.0)


In [233]:
## 9월달 일 데이터 (기온, 풍속, 깅수량)

w1=weather_9m.groupby(['지점','일시'])[['기온(°C)','풍속(m/s)']].mean()
w2=weather_9m.groupby(['지점','일시'])['강수량(mm)'].sum()  #일강수량
weather_9_avg=pd.merge(w1,w2, how='outer',on=['지점','일시'])
weather_9_avg

weather_9_avg.to_csv('weather_9_avg')

기온(°C)   풍속(m/s)  강수량(mm)
지점       일시                                      
gosan    2019-09-01  22.766667  2.612500      8.8
         2019-09-02  24.595833  3.729167     53.5
         2019-09-03  24.579167  3.145833     39.3
         2019-09-04  24.304167  4.704167     67.9
         2019-09-05  26.383333  4.633333      0.0
...                        ...       ...      ...
seongsan 2019-09-26  24.062500  4.233333      0.0
         2019-09-27  23.854167  3.137500     70.4
         2019-09-28  24.945833  1.754167     11.2
         2019-09-29  24.358333  1.550000      2.5
         2019-09-30  23.941667  1.358333      3.9

[120 rows x 3 columns]

In [226]:
#10월데이터(오전)
weather_10m= weather[(weather['일시']>='2019-10-01') ]
weather_10m=weather_10m[(weather_10m['time']>=6) & (weather_10m['time']<12)]

weather_10m=weather_10m[cols]

#na 값에 0.0
weather_10m=weather_10m.fillna(0.0)
weather_10m.head(3)

,지점,일시,time,기온(°C),강수량(mm),풍속(m/s),습도(%),전운량(10분위),현상번호(국내식)
726,jeju,2019-10-01,6,23.8,0.4,1.4,80,10.0,1904.0
727,jeju,2019-10-01,7,22.2,0.7,1.7,90,10.0,1904.0
728,jeju,2019-10-01,8,24.5,0.0,3.1,92,10.0,0.0


In [240]:
## 10월달 오전 데이터(6~12시전) (기온, 풍속, 오전 일깅수량)
b1=weather_10m.groupby(['지점','일시'])[['기온(°C)','풍속(m/s)']].mean()
b2=weather_10m.groupby(['지점','일시'])['강수량(mm)'].sum()  #일강수량
weather_10_morning=pd.merge(b1,b2, how='outer',on=['지점','일시'])
weather_10_morning.to_csv('weather_10_morning')


기온(°C)    풍속(m/s)  강수량(mm)
지점       일시                                       
gosan    2019-10-01  23.883333   4.433333      1.5
         2019-10-02  25.100000   9.300000     33.3
         2019-10-03  23.133333  14.516667      0.3
         2019-10-04  23.233333   5.566667      0.0
         2019-10-05  21.666667   6.816667      0.0
...                        ...        ...      ...
seongsan 2019-10-12  21.300000   4.633333      0.0
         2019-10-13  19.300000   4.466667      0.0
         2019-10-14  18.583333   2.500000      0.0
         2019-10-15  16.216667   3.333333      2.9
         2019-10-16  18.883333   3.966667      0.0

[64 rows x 3 columns]

In [241]:
#10월데이터(오후)
weather_10_aft= weather[(weather['일시']>='2019-10-01') ]
weather_10_aft=weather_10_aft[(weather_10_aft['time']>=12) &(weather_10_aft['time']<=23)]
                                                            
weather_10_aft=weather_10_aft[cols]


#na 값에 0.0
weather_10_aft=weather_10_aft.fillna(0.0)
weather_10_aft


In [235]:
## 10월달 오후 데이터(12~23시) (기온, 풍속, 오후 일깅수량)
a1=weather_10_aft.groupby(['지점','일시'])[['기온(°C)','풍속(m/s)']].mean()
a2=weather_10_aft.groupby(['지점','일시'])['강수량(mm)'].sum()  #일강수량
weather_10_noon=pd.merge(a1,a2, how='outer',on=['지점','일시'])

weather_10_noon.to_csv('weather_10_noon')
